In [1]:
import numpy as np; np.set_printoptions(precision=3)
import numpy.random as rnd
import matplotlib.pyplot as plt

def get_super_condition(a_1=None):
    # Condicao:
    #   a_1^2-4*a_0 > 0
    #     a_0 < a_1^2/4
    a_0 = (a_1**2)/4
    print(f'Superamortecido:\n  a_0 < {a_0:.3f}')
    return a_0

def get_crit_condition(a_1=None):
    # Condicao:
    #   a_1^2-4*a_0 = 0
    #     a_0 = a_1^2/4
    a_0 = a_1**2/4
    print(f'Criticamente amortecido:\n  a_0 = {a_0:.3f}')
    return a_0

def get_sub_condition(a_1=None):
    # Condicao:
    #   a_1^2-4*a_0 < 0
    #     a_0 > a_1^2/4
    a_0 = a_1**2/4
    print(f'Subamortecido:\n  a_0 > {a_0:.3f}')
    return a_0

def get_system_types(a_1=None):    
    a_0_sup = get_super_condition(a_1)
    a_0_crit = get_crit_condition(a_1)
    a_0_sub = get_sub_condition(a_1)
    return a_0_sup, a_0_crit, a_0_sub
    
def get_characteristic(matrix,verbose=False):
    """[summary]

    Args:
        A (np.array[(n,n)]): Matriz de estados
        verbose (bool, optional): Aumenta a verbosidade da função. Por padrão é False.

    Returns:
        np.array: Lista de coeficientes da função característica.
    """    
    base=-np.ones((1,matrix.shape[0]))[0]
    multi = np.linspace(matrix.shape[0],1,matrix.shape[0])
    tr = np.trace(matrix)
    det = np.linalg.det(matrix)
    res = np.concatenate((base**multi,[det]))
    res[1:-1] = res[1:-1]*tr
    res = -1*res if res[0]<0 else res
    if verbose: print("res =",res)
    return res

def get_eigenval(matrix,verbose=False):
    """Calcula os autovalores relacionados à matriz de estados.

    Args:
        A (np.array[(n,n)]): Matriz de estados
        verbose (bool, optional): Aumenta a verbosidade da função. Por padrão é False.

    Returns:
        np.array: Lista de autovalores.
    """    
    return np.roots(get_characteristic(matrix))

def get_eigenvec(matrix,verbose=False):
    """Calcula os autovetores relacionados à matriz de estados.

    Args:
        A (np.array[(n,n)]): Matriz de estados
        verbose (bool, optional): Aumenta a verbosidade da função. Por padrão é False.

    Returns:
        np.array: Lista de autovetores.
    """    
    # Get eigenvalues
    eigval = get_eigenval(matrix)
   
    temp = np.tensordot(eigval,np.identity(matrix.shape[0]),0)-matrix
    temp_b = np.zeros((1,temp.shape[-1])).T
    
    eigvec = []
    for idx_a, a in enumerate(temp):
        normalized = np.array([b/b[0] for b in a])
        if verbose: print("  Normalized = \n",normalized)
        li_comparison = np.all(np.array([np.isclose(normalized[0],b,rtol=1e-10) for b in normalized])==True)
        if verbose: print("  Is L.I.?",li_comparison)
        if(li_comparison): 
            eig = np.array([[1],[-a[0,0]/a[0,1]]])
        else: 
            eig = np.array(np.linalg.solve(a,temp_b))
        eigvec.append(eig)
    eigvec = np.array(eigvec)
    return eigvec

def get_system(A,X_init=None):
    """Calcula autovalores, autovetores e a matriz de amplitudes,
    com base na matriz de estados e nas condições iniciais.

    Args:
        A (np.array[(n,n)]): Matriz de estados
        X_init (np.array[(2,1)], optional): Matriz de condições iniciais. Por padrão 
        tem valor None, que inicializa as condições iniciais com valor unitário.

    Returns:
        np.array: Matriz de amplitudes.
    """    
    eigenval =  get_eigenval(A)
    eigenvec =  get_eigenvec(A)
    if(np.any(X_init==None)): X_init = np.ones_like(eigenval)
    C = np.linalg.inv(np.concatenate(eigenvec,axis=1))@X_init
    return C


# Exemplos

## Exemplo 3.1

In [2]:
A = np.array([[0,1],
              [-0.9,-2]])
print("A =\n", A,"\n")
autoval = get_eigenval(A)
print("Autovalor de A =",autoval,"\n")
autovec = get_eigenvec(A)
print(f"Autovetor de A(Shape: {autovec.shape}) =\n{autovec}","\n")
X = np.ones_like(autoval)
print(f"X =\n{X}","\n")
C = get_system(A,X)
print(f"C =\n{C}","\n")
X_dot=A@X
print(f"X_dot =\n{X_dot}","\n")

A =
 [[ 0.   1. ]
 [-0.9 -2. ]] 

Autovalor de A = [-1.316 -0.684] 

Autovetor de A(Shape: (2, 2, 1)) =
[[[ 1.   ]
  [-1.316]]

 [[ 1.   ]
  [-0.684]]] 

X =
[1. 1.] 

C =
[-2.662  3.662] 

X_dot =
[ 1.  -2.9] 



## Exemplo 3.2

In [3]:
A = np.array([[0,1],
              [-1,0]])
print("A =\n", A,"\n")
autoval = get_eigenval(A)
print("Autovalor de A =",autoval,"\n")
autovec = get_eigenvec(A)
print(f"Autovetor de A(Shape: {autovec.shape}) =\n{autovec}","\n")
X = np.ones_like(autoval)
print(f"X =\n{X}","\n")
C = get_system(A,X)
c_1 = np.real(C)[1]
c_2 = np.imag(C)[1]
print(f"C =\n{C}\nc_1 = {c_1}\nc_2 = {c_2}","\n")
X_dot=A@X
print(f"X_dot =\n{X_dot}","\n")

A =
 [[ 0  1]
 [-1  0]] 

Autovalor de A = [0.+1.j 0.-1.j] 

Autovetor de A(Shape: (2, 2, 1)) =
[[[ 1.+0.j]
  [-0.+1.j]]

 [[ 1.+0.j]
  [ 0.-1.j]]] 

X =
[1.+0.j 1.+0.j] 

C =
[0.5-0.5j 0.5+0.5j]
c_1 = 0.5
c_2 = 0.5 

X_dot =
[ 1.+0.j -1.+0.j] 



## Exemplo 3.3

In [4]:
A = np.array([[0,1],
              [-0.9,-2]])
print("A =\n", A,"\n")
autoval = get_eigenval(A)
print("Autovalor de A =",autoval,"\n")
autovec = get_eigenvec(A)
print(f"Autovetor de A(Shape: {autovec.shape}) =\n{autovec}","\n")
X = np.ones_like(autoval)
print(f"X =\n{X}","\n")
C = get_system(A,X)
print(f"C =\n{C}","\n")
X_dot=A@X
print(f"X_dot =\n{X_dot}","\n")

A =
 [[ 0.   1. ]
 [-0.9 -2. ]] 

Autovalor de A = [-1.316 -0.684] 

Autovetor de A(Shape: (2, 2, 1)) =
[[[ 1.   ]
  [-1.316]]

 [[ 1.   ]
  [-0.684]]] 

X =
[1. 1.] 

C =
[-2.662  3.662] 

X_dot =
[ 1.  -2.9] 



# Exercício

In [5]:
rnd.seed(11025215)
a_1 = -(1+rnd.randint(1,100)/100)
X_init = np.array([rnd.randint(1,4)+rnd.randint(1,100)/100,rnd.randint(1,4)+rnd.randint(1,100)/100])
print(f"X_init = {X_init}\na_1 = {a_1}")
_, a_0_crit,_ = get_system_types(a_1=a_1)

a_0_sup = 0.525
a_0_sub = 0.890

X_init = [1.1  1.68]
a_1 = -1.74
Superamortecido:
  a_0 < 0.757
Criticamente amortecido:
  a_0 = 0.757
Subamortecido:
  a_0 > 0.757


## Superamortecido($a_0<13.69$)

In [6]:
A = np.array([[0,1],
              [-a_0_sup,-a_1]])
print("A =\n", A,"\n")
autoval = get_eigenval(A)
print("Autovalor de A =",autoval,"\n")
autovec = get_eigenvec(A)
print(f"Autovetor de A(Shape: {autovec.shape}) =\n{autovec}","\n")
X = X_init
print(f"X =\n{X}","\n")
C = get_system(A,X)
print(f"C =\n{C}","\n")
X_dot=A@X
print(f"X_dot =\n{X_dot}","\n")

A =
 [[ 0.     1.   ]
 [-0.525  1.74 ]] 

Autovalor de A = [1.352 0.388] 

Autovetor de A(Shape: (2, 2, 1)) =
[[[1.   ]
  [1.352]]

 [[1.   ]
  [0.388]]] 

X =
[1.1  1.68] 

C =
[ 1.301 -0.201] 

X_dot =
[1.68  2.346] 



## Criticamente amortecido($a_0=13.69$)

In [7]:
A = np.array([[0,1],
              [-a_0_crit,-a_1]])
print("A =\n", A,"\n")
autoval = get_eigenval(A)
print("Autovalor de A =",autoval,"\n")
autovec = get_eigenvec(A)
print(f"Autovetor de A(Shape: {autovec.shape}) =\n{autovec}","\n")
X = X_init
print(f"X =\n{X}","\n")
C = get_system(A,X)
print(f"C =\n{C}","\n")
X_dot=A@X
print(f"X_dot =\n{X_dot}","\n")

A =
 [[ 0.     1.   ]
 [-0.757  1.74 ]] 

Autovalor de A = [0.87 0.87] 

Autovetor de A(Shape: (2, 2, 1)) =
[[[1.  ]
  [0.87]]

 [[1.  ]
  [0.87]]] 

X =
[1.1  1.68] 

C =
[ 36605359.001 -36605357.901] 

X_dot =
[1.68  2.091] 



## Subamortecido($a_0>13.69$)

In [8]:
A = np.array([[0,1],
              [-a_0_sub,-a_1]])
print("A =\n", A,"\n")
autoval = get_eigenval(A)
print("Autovalor de A =",autoval,"\n")
autovec = get_eigenvec(A)
print(f"Autovetor de A(Shape: {autovec.shape}) =\n{autovec}","\n")
X = X_init
print(f"X =\n{X}","\n")
C = get_system(A,X)
c_1 = np.real(C)[1]
c_2 = np.imag(C)[1]
print(f"C =\n{C}\nc_1 = {c_1:.3f}\nc_2 = {c_2:.3f}","\n")
X_dot=A@X
print(f"X_dot =\n{X_dot}","\n")

A =
 [[ 0.    1.  ]
 [-0.89  1.74]] 

Autovalor de A = [0.87+0.365j 0.87-0.365j] 

Autovetor de A(Shape: (2, 2, 1)) =
[[[1.  +0.j   ]
  [0.87+0.365j]]

 [[1.  +0.j   ]
  [0.87-0.365j]]] 

X =
[1.1  1.68] 

C =
[0.55-0.991j 0.55+0.991j]
c_1 = 0.550
c_2 = 0.991 

X_dot =
[1.68  1.944] 

